In [ ]:
from music21 import *

environment.UserSettings()['lilypondPath']='/usr/bin/lilypond'
environment.set('pdfPath', '/usr/bin/musescore')
environment.set('graphicsPath', '/usr/bin/musescore')
environment.set("musescoreDirectPNGPath", "/usr/bin/musescore")
environment.set("musicxmlPath", "/usr/bin/musescore")

from IPython.display import Image

In [ ]:
c = chord.Chord("C4 E4 G4")
c.isConsonant()
c.show('lily.png')
Image(filename=c.write('lily.png'))

In [ ]:
myBach = corpus.parse('bach/bwv57.8')
alto = myBach.parts['Alto']

alto.show('lily.png')
Image(filename=alto.write('lily.png'))